<a href="https://colab.research.google.com/github/Investigator13th/CS246/blob/main/CS246_Colab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 7
## Decision Trees in Spark

In [1]:
from IPython.display import Image
print("Colab 7 Mascot")
Image(url='https://media2.giphy.com/media/kudIERso2pFiE/giphy.gif?cid=ecf05e47bipq7szjsc8vmss2uxmt1zha9gksgdrn1z8mmadj&ep=v1_gifs_search&rid=giphy.gif&ct=g',width=200)

Colab 7 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=deabd2cc5cb53e0691324cb126c6a8bdca0e793f5894b493e1cd76a6d2d21818
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1aJrdYMVmmnUKYhLTlXtyB0FQ9gYJqCrs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1yLwxRaJIyrC03yxqbTKpedMmHEF86AAq'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf() \
    .set("spark.ui.port", "4050") \
    .set("spark.driver.cores", "4") \
    .set("spark.driver.memory", "16g") \
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

![MNIST](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/220px-MnistExamples.png)

In this Colab, we will be using the famous [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), a large collection of handwritten digits that is widely used for training and testing in the field of machine learning.

For your convenience, the dataset has already been converted to the popular LibSVM format, where each digit is represented as a sparse vector of grayscale pixel values.

In [7]:
training = spark.read.format("libsvm").load("mnist-digits-train.txt")
test = spark.read.format("libsvm").load("mnist-digits-test.txt")

# Cache data for multiple uses
training.cache()
test.cache()

DataFrame[label: double, features: vector]

In [8]:
training.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
training.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
test.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



### Your task

First of all, find out how many instances(samples) we have in our training / test split.

In [14]:
''' 2 lines of code in total expected. '''

# YOUR CODE HERE
print(training.count())
print(test.count())

60000
10000


Now train a Decision Tree on the training dataset using Spark MLlib.

You can refer to the Python example on this documentation page: [https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)

(Note : You **don't** need to and should **NOT** use Indexer as shown by the example, so that your answers can match our solution answers for Gradescope problems.)

Use **seed of 7** (colab 7) for training purposes throughout the colab.

In [17]:
''' 7-8 lines (this number includes package import) of code in total expected but can differ based on your style.  '''
''' For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(training)

With the Decision Tree you just induced on the training data, predict the labels of the test set.
Print the predictions for the first 10 digits, and compare them with the labels.

In [20]:
''' 2 lines of code in total expected. '''

# YOUR CODE HERE
predictions = model.transform(test)
predictions.select('label').take(10)

[Row(label=7.0),
 Row(label=2.0),
 Row(label=1.0),
 Row(label=0.0),
 Row(label=4.0),
 Row(label=1.0),
 Row(label=4.0),
 Row(label=9.0),
 Row(label=5.0),
 Row(label=9.0)]

In [21]:
test.select('label').take(10)

[Row(label=7.0),
 Row(label=2.0),
 Row(label=1.0),
 Row(label=0.0),
 Row(label=4.0),
 Row(label=1.0),
 Row(label=4.0),
 Row(label=9.0),
 Row(label=5.0),
 Row(label=9.0)]

The small sample above looks good, but not great!

Let's dig deeper. Compute the accuracy of our model, using the ```MulticlassClassificationEvaluator``` from MLlib.

In [23]:
''' 4-5 lines of code in total expected. '''

# YOUR CODE HERE
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.3223


Find out the max depth of the trained Decision Tree, and its total number of nodes.

In [24]:
''' 1 line of code in total expected. '''

# YOUR CODE HERE
print(f'model.depth:{model.depth}')
print(f'model.depth:{model.numNodes}')

model.depth:5
model.depth:59


It appears that the default settings of the Decision Tree implemented in MLlib did not allow us to train a very powerful model!

Before starting to train a Decision Tree, you can tune the max depth it can reach using the [setMaxDepth()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html#pyspark.ml.classification.DecisionTreeClassifier.setMaxDepth) method. Train 21 different DTs, varying the max depth from 0 to 20, endpoints included (i.e., [0, 20]). For each value of the parameter, print the accuracy achieved on the test set, and the number of nodes contained in the given DT.

For this problem, we say that the decision trees overfit the dataset if the classification accuracy starts to drop.

**IMPORTANT:** this parameter sweep can take 30 minutes or more, depending on how busy is your Colab instance. Notice how the induction time grows super-linearly!

In [26]:
''' 6-7 lines of code in total expected. '''
''' For sub-parts of the question (if any), creating different cells of code would be recommended.'''

# YOUR CODE HERE
def train_and_evaluate(dt_max_depth, training_data, test_data):
  # 创建决策树分类器
  dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=dt_max_depth)

  # 训练模型
  model = dt.fit(training_data)

  # 使用模型进行预测
  predictions = model.transform(test_data)

  # 评估模型
  evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
  accuracy = evaluator.evaluate(predictions)

  # 打印精度和节点数量
  print(f"Max Depth: {dt_max_depth}, Accuracy: {accuracy}, Number of Nodes: {model.numNodes}")

In [ ]:
for max_depth in range(21):  # 包括0到20
    train_and_evaluate(max_depth, training, test)

Max Depth: 0, Accuracy: 0.1135, Number of Nodes: 1
Max Depth: 1, Accuracy: 0.1994, Number of Nodes: 3
Max Depth: 2, Accuracy: 0.3457, Number of Nodes: 7
Max Depth: 3, Accuracy: 0.4975, Number of Nodes: 15
Max Depth: 4, Accuracy: 0.6047, Number of Nodes: 31
Max Depth: 5, Accuracy: 0.6777, Number of Nodes: 59
Max Depth: 6, Accuracy: 0.7481, Number of Nodes: 119
Max Depth: 7, Accuracy: 0.7933, Number of Nodes: 247
Max Depth: 8, Accuracy: 0.822, Number of Nodes: 477
Max Depth: 9, Accuracy: 0.8512, Number of Nodes: 937
Max Depth: 10, Accuracy: 0.8704, Number of Nodes: 1715


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!